# Projeto 3: Operando no MetaTrader e comprando as ações indicadas pelo modelo.

### Desafio:

* Construir um código que se conecte direto na bolsa e compre as ações indicadas.
   
### Passo a passo do código? 

   **Passo 1** - Rodar o código da aula anterior.
   
   **Passo 2** - Baixar e configurar o MetaTrader pra sua conta em uma corretora.
   
   **Passo 3** - Importar as bibliotecas. 

   **Passo 4** - Inicializar o MetaTrader. 
   
   **Passo 5** - Configurar o dicionário da ordem e comprar ações.

# Passo 1: Rodar o código da aula anterior. 

In [5]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager

# Configura o serviço do Edge WebDriver
service = Service(EdgeChromiumDriverManager().install())

# Inicializa o driver do Edge
driver = webdriver.Edge(service=service)

In [6]:


url = 'https://www.fundamentus.com.br/resultado.php'

driver.get(url)
local_tabela = '/html/body/div[1]/div[2]/table'
elemento = driver.find_element("xpath", local_tabela)
html_tabela = elemento.get_attribute('outerHTML')
tabela = pd.read_html(str(html_tabela), thousands = '.')[0]

tabela = tabela.set_index("Papel")
for col in tabela.columns:

    tabela[col] = tabela[col].str.replace("%", "")
    tabela[col] = tabela[col].str.replace(".", "")
    tabela[col] = tabela[col].str.replace(",", ".")
    tabela[col] = tabela[col].astype(float)

tabela = tabela[['Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses']]
tabela = tabela[tabela['Liq.2meses'] > 1000000]
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]

tabela['ranking_ev_ebit'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['ranking_roic'] = tabela['ROIC'].rank(ascending = False)
tabela['ranking_total'] = tabela['ranking_ev_ebit'] + tabela['ranking_roic']
tabela = tabela.sort_values('ranking_total')
driver.quit()

C:\Users\pichau\AppData\Local\Temp\ipykernel_19832\1712327767.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabela = pd.read_html(str(html_tabela), thousands = '.')[0]


In [7]:
tabela = tabela.head(10)

tickers = tabela.index

tickers

Index(['PSSA3', 'WIZC3', 'PLPL3', 'CMIN3', 'PETR4', 'PETR3', 'VLID3', 'VALE3',
       'LREN3', 'LEVE3'],
      dtype='object', name='Papel')

# Passo 2: Baixar e configurar o MT5.

* Cada corretora é de uma forma. Pesquise como é feito na sua.
* Conta demo, sempre!

# Passo 3: Importar a biblioteca.

In [10]:
import MetaTrader5 as mt5


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\pichau\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\pichau\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\pichau\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\kernelapp.py

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

# Passo 4: Inicializar o MT5.

In [ ]:
mt5.initialize()

# Passo 6: Configurar o dicionário da ordem e comprar ações.

https://www.mql5.com/en/docs/python_metatrader5

In [ ]:
for ticker in tickers:

    print(ticker)

    info_acoes = mt5.symbol_info(ticker)
    mt5.symbol_select(ticker)
    tick_min = mt5.symbol_info(ticker).point
    preco = mt5.symbol_info_tick(ticker).ask
    quantidade = 100.0
    ordem_compra = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": quantidade,
        "type": mt5.ORDER_TYPE_BUY,
        "price": preco,
        "magic": 1,
        "comment": "Trades automáticos",
        "type_time": mt5.ORDER_TIME_DAY,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result_compra = mt5.order_send(ordem_compra)